In [1]:
!pip install surprise

In [4]:
import pandas as pd
import requests, zipfile
import numpy as np
import tqdm

In [17]:
#!pip install --upgrade numpy
!pip install 'numpy<2.0'

In [6]:
import surprise
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

In [5]:
np.__version__

'1.26.4'

In [ ]:
#!wget 'https://drive.google.com/uc?id=1EzhdmPqtnVLRGAk8EXTNfq-beb_-3CUz' -O RS_movies_small.zip # 1 вариант

In [ ]:
# !unzip -o RS_movies_small.zip # 1 вариант
# !unzip -o ml-latest-small.zip или !yes A | unzip ml-latest-small.zip (архив автоматически распаковывается с заменой всех файлов без запроса подтверждения)

In [7]:
response = requests.get('https://drive.google.com/uc?id=1EzhdmPqtnVLRGAk8EXTNfq-beb_-3CUz')

In [8]:
with open('RS_movies_small.zip', mode='wb') as zip_:
  zip_.write(response.content)

In [9]:
with zipfile.ZipFile('RS_movies_small.zip', mode='r') as data_files:
  data_files.extractall()

In [10]:
tags = pd.read_csv('tags.csv')
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
links = pd.read_csv('links.csv')

In [11]:
movies_with_ratings = ratings.merge(right=movies, on='movieId', how='inner') # how='right' - с целью сохранения фильмов где нет рейтинга (18 записей)

In [13]:
movies_with_ratings.head(3)

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller


In [15]:
movie_vector = {}
for movie, group in movies_with_ratings.groupby('title'):
  movie_vector[movie] = np.zeros(movies_with_ratings['userId'].nunique())

In [16]:
for mov in movies_with_ratings['movieId'].unique():
  vector = np.full(mov, np.nan) # код выполнен неполностью

In [14]:
# вариант с формированием сводной таблицы.
matrix = movies_with_ratings.pivot_table(index='movieId',
    columns='userId',
    values='rating',
    aggfunc='first').fillna(0)

In [15]:
matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
dict_movie_vector={}

In [17]:
for movie_id in tqdm.tqdm(matrix.index):
  dict_movie_vector.setdefault(movie_id, matrix.loc[movie_id].values)

100%|██████████| 9724/9724 [00:00<00:00, 43045.43it/s]


In [22]:
dict_movie_vector[1][:20] # первые 20 оценок для фильма с ID=1

array([4. , 0. , 0. , 0. , 4. , 0. , 4.5, 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 2.5, 0. , 4.5, 3.5, 4. , 0. ])

In [24]:
np.argsort(dict_movie_vector[1][:20]) # Сортировка и указывает индексы оценок

array([ 9, 15, 13, 12, 11, 10,  8, 19,  5,  3,  2,  1,  7, 14, 17, 18,  4,
        0,  6, 16])

## **Surprise**

In [25]:
data = movies_with_ratings[['title', 'userId', 'rating']]
print(f'shape:{data.shape}')
print(f'Уникальных фильмов: {data.title.nunique()}\nУникальных пользователей: {data.userId.nunique()}')
data.head(3)

shape:(100836, 3)
Уникальных фильмов: 9719
Уникальных пользователей: 610


,title,userId,rating
0,Toy Story (1995),1,4.0
1,Grumpier Old Men (1995),1,4.0
2,Heat (1995),1,4.0


In [26]:
reader_surprise = Reader(rating_scale=(np.min(data['rating']), np.max(data['rating'])))

In [27]:
data_surprise = Dataset.load_from_df(df=data, reader=reader_surprise)
data_surprise

**МОДЕЛИ**

In [28]:
print('Similarity measure configuration (sim_options) - Конфигурация меры подобия'.upper())

SIMILARITY MEASURE CONFIGURATION (SIM_OPTIONS) - КОНФИГУРАЦИЯ МЕРЫ ПОДОБИЯ


In [ ]:
"""
'name': 'cosine', 'pearson', 'msd' | метрики сходства между элементами или пользователями
'user_based': False                | True: Режим user-based - находит похожих пользователей и использует их оценки для предсказания.
                                   | False: Режим item-based - находит похожие элементы (фильмы) и использует оценки текущего пользователя для похожих элементов.
'min_support: 5'                   | Выбрал минимум 5 общих оценок
'shrinkage': 100                   | Применяет сильную регуляризацию, 0 - регуляризация не применяется. Помогает бороться с низкой достоверностью коэффициентов корреляции при малом количестве общих оценок.
"""

**user_based**

In [89]:
models_surprise = [surprise.KNNBasic(k=50, min_k=1, sim_options={'name': 'cosine', 'user_based': False, 'shrinkage': 0}),
                   surprise.KNNWithMeans(k=50, min_k=1, sim_options={'name': 'cosine', 'user_based': False}),
                   surprise.SVD(),
                   surprise.SVDpp(),
                   surprise.SlopeOne()
                   ]

In [90]:
results_models = {}

In [91]:
# Проведение 5-фолдовой кросс-валидации | cv=5
for model in models_surprise:
  cross_validate_model = surprise.model_selection.cross_validate(algo=model, data=data_surprise, measures=['rmse', 'mae'], cv=5, verbose=True)

  mean_rmse = np.mean(cross_validate_model['test_rmse'])
  mean_mae = np.mean(cross_validate_model['test_mae'])
  mean_fit_time = np.mean(cross_validate_model['fit_time'])
  mean_test_time = np.mean(cross_validate_model['test_time'])

  results_models.setdefault(model.__class__.__name__, [mean_rmse, mean_mae, mean_fit_time, mean_test_time])


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9734  0.9671  0.9660  0.9865  0.9716  0.9729  0.0073  
MAE (testset)     0.7500  0.7467  0.7439  0.7597  0.7503  0.7501  0.0053  
Fit time          0.18    0.29    0.21    0.16    0.17    0.20    0.05    
Test time         2.00    1.36    1.27    1.48    1.23    1.47    0.28    
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity ma

In [92]:
results_models

{'KNNBasic': [0.9729194383036182,
  0.750094215230596,
  0.20377039909362793,
  1.4707654953002929],
 'KNNWithMeans': [0.9005894970339593,
  0.6889205769477281,
  0.1922219753265381,
  1.5509285926818848],
 'SVD': [0.8780025888265992,
  0.6750640355842592,
  1.5393243312835694,
  0.171337890625],
 'SVDpp': [0.8465133217125775,
  0.6504988900846278,
  8.441172695159912,
  1.8432910919189454],
 'SlopeOne': [0.8903604738052877,
  0.6796974669674808,
  0.26143059730529783,
  0.8806300640106202]}

In [93]:
# сортировка словаря
results = sorted(results_models.items(), key=lambda key: key[1], reverse=False)
results

[('SVDpp',
  [0.8465133217125775,
   0.6504988900846278,
   8.441172695159912,
   1.8432910919189454]),
 ('SVD',
  [0.8780025888265992,
   0.6750640355842592,
   1.5393243312835694,
   0.171337890625]),
 ('SlopeOne',
  [0.8903604738052877,
   0.6796974669674808,
   0.26143059730529783,
   0.8806300640106202]),
 ('KNNWithMeans',
  [0.9005894970339593,
   0.6889205769477281,
   0.1922219753265381,
   1.5509285926818848]),
 ('KNNBasic',
  [0.9729194383036182,
   0.750094215230596,
   0.20377039909362793,
   1.4707654953002929])]

In [94]:
# Для удобства, формирование датафрейма и сортировка
df_results_models = pd.DataFrame(results_models, index=['RMSE', 'MAE', 'fit_time', 'test_time']).sort_index()
df_results_models

,KNNBasic,KNNWithMeans,SVD,SVDpp,SlopeOne
MAE,0.750094,0.688921,0.675064,0.650499,0.679697
RMSE,0.972919,0.900589,0.878003,0.846513,0.890360
fit_time,0.203770,0.192222,1.539324,8.441173,0.261431
test_time,1.470765,1.550929,0.171338,1.843291,0.880630
